# Test Pipeline
Objective: Test training pipeline

## 1. Setup and imports

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Explicit .env loading for Jupyter (Docker works, Jupyter needs this)
env_file = Path("../.env")  # .env is in project root
if env_file.exists():
    load_dotenv(env_file)
    print(".env file loaded successfully")
    print(f"DB_PASSWORD length: {len(os.getenv('DB_PASSWORD', ''))} chars")
else:
    print(".env file not found at ../.env")
    print("Current working directory:", os.getcwd())

.env file loaded successfully
DB_PASSWORD length: 20 chars


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID", "test")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY", "test")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION", "us-east-1")
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("AWS_ENDPOINT_URL", "http://localhost:4566")

print("MLflow S3 environment configured:")
print(f"   S3 Endpoint: {os.environ['MLFLOW_S3_ENDPOINT_URL']}")

MLflow S3 environment configured:
   S3 Endpoint: http://localhost:4566


In [3]:
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime

import mlflow

warnings.filterwarnings("ignore")

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
from src.data.preprocessor import SolarForecastingPreprocessor
from src.model.model_config import get_training_config
from src.model.trainer import ModelTrainer
from src.model.evaluator import ModelEvaluator
from src.model.registry import ModelRegistry

In [5]:
temp_preprocessor = SolarForecastingPreprocessor(
    forecast_horizon=24,
    lag_days=[1, 2, 3, 7, 30],
    rolling_windows=[7, 30],
    scaling_method='standard'
)

X_full, y_full, metadata = temp_preprocessor.fit_transform(
    "../data/raw/Plant_1_Generation_Data.csv",
    "../data/raw/Plant_1_Weather_Sensor_Data.csv"
)

split_point = int(0.8 * len(X_full))
X_test = X_full.iloc[split_point:].reset_index(drop=True)
y_test = y_full.iloc[split_point:].reset_index(drop=True)

data, _ = temp_preprocessor.load_and_prepare_data(
    "../data/raw/Plant_1_Generation_Data.csv",
    "../data/raw/Plant_1_Weather_Sensor_Data.csv"
)

INFO:src.data.preprocessor:Initialized SolarForecastingPreprocessor: horizon=24h, lags=[1, 2, 3, 7, 30] days, rolling_windows=[7, 30] days, scaling=standard
INFO:src.data.preprocessor:Starting complete forecasting preprocessing pipeline...
INFO:src.data.preprocessor:Loading solar generation data...
INFO:src.data.preprocessor:Resampling to hourly frequency...
INFO:src.data.preprocessor:Removed 20 NaN rows after resampling
INFO:src.data.preprocessor:Data loaded successfully: 796 records from 2020-05-15 00:00:00 to 2020-06-17 23:00:00
INFO:src.data.preprocessor:Validating forecasting setup for data leakage...
INFO:src.data.preprocessor:✅ Forecasting setup validation PASSED
INFO:src.data.preprocessor:Creating forecasting dataset...
INFO:src.data.preprocessor:Creating historical lag features...
INFO:src.data.preprocessor:Created 5 lag features and 2 rolling features
INFO:src.data.preprocessor:Creating temporal features for future dates...
INFO:src.data.preprocessor:Created 16 temporal featu

## 2. Training

In [6]:
config = get_training_config(
    generation_data_path=None,
    weather_data_path=None
)
config.to_dict()

{'model_config': {'n_estimators': 50,
  'max_depth': 4,
  'learning_rate': 0.1,
  'subsample': 0.8,
  'colsample_bytree': 0.8,
  'random_state': 42,
  'n_jobs': -1,
  'verbosity': 0},
 'validation_config': {'n_splits': 3, 'test_size': 7, 'min_train_size': 20},
 'preprocessor_config': {'forecast_horizon': 24,
  'lag_days': [1, 2, 3, 7, 30],
  'rolling_windows': [7, 30],
  'scaling_method': 'standard',
  'target_frequency': '1H'},
 'experiment_name': 'solar-forecasting-production',
 'run_name_prefix': 'xgboost_training',
 'generation_data_path': '/workspaces/solar-forecasting-mlops/data/raw/Plant_1_Generation_Data.csv',
 'weather_data_path': '/workspaces/solar-forecasting-mlops/data/raw/Plant_1_Weather_Sensor_Data.csv'}

In [7]:
trainer = ModelTrainer(config)

INFO:src.data.preprocessor:Initialized SolarForecastingPreprocessor: horizon=24h, lags=[1, 2, 3, 7, 30] days, rolling_windows=[7, 30] days, scaling=standard
INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.model.trainer:MLflow tracking URI: http://localhost:5000
INFO:src.model.trainer:S3 Endpoint: http://localhost:4566
INFO:src.model.trainer:ModelTrainer initialized with config: solar-forecasting-production
INFO:src.model.trainer:Model params: {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 42, 'n_jobs': -1, 'verbosity': 0}


In [8]:
model, metrics, run_id = trainer.train()  

INFO:src.model.trainer:Starting complete training pipeline...
INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.utils.mlflow_utils:Using existing active experiment: solar-forecasting-production
INFO:src.model.trainer:Loading and preprocessing data...
INFO:src.data.preprocessor:Starting complete forecasting preprocessing pipeline...
INFO:src.data.preprocessor:Loading solar generation data...
INFO:src.data.preprocessor:Resampling to hourly frequency...
INFO:src.data.preprocessor:Removed 20 NaN rows after resampling
INFO:src.data.preprocessor:Data loaded successfully: 796 records from 2020-05-15 00:00:00 to 2020-06-17 23:00:00
INFO:src.data.preprocessor:Validating forecasting setup for data leakage...
INFO:src.data.preprocessor:✅ Forecasting setup validation PASSED
INFO:src.data.preprocessor:Creating forecasting dataset...
INFO:src.data.preprocessor:Creating historical lag

🏃 View run xgboost_training_20250729_131844 at: http://localhost:5000/#/experiments/3/runs/770d909ddaf74e91ace9011fd3c09eb9
🧪 View experiment at: http://localhost:5000/#/experiments/3


In [9]:
model

,estimator,"XGBRegressor(...ree=None, ...)"
,n_jobs,-1
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None


In [10]:
metrics

{'cv_rmse_mean': np.float64(174.51795206302867),
 'cv_rmse_std': np.float64(15.370274465174798),
 'cv_mae_mean': np.float64(110.11674873293363),
 'cv_mae_std': np.float64(6.018690906301035),
 'cv_r2_mean': np.float64(0.7071791847697418),
 'cv_r2_std': np.float64(0.03885596264813393),
 'train_rmse_overall': np.float64(14.292465009868055),
 'train_mae_overall': 7.417960157318693,
 'train_r2_overall': 0.9980629386647818,
 'train_rmse_1h': np.float64(12.112551849856395),
 'train_mae_1h': 6.259222121121651,
 'train_rmse_6h': np.float64(12.719038540848043),
 'train_mae_6h': 6.487942083073468,
 'train_rmse_12h': np.float64(14.594851327805253),
 'train_mae_12h': 9.21077878285092,
 'train_rmse_24h': np.float64(18.564160827986957),
 'train_mae_24h': 5.862646503051318,
 'train_rmse_horizon_mean': np.float64(14.188855488244798),
 'train_rmse_horizon_std': np.float64(1.7178288599299159),
 'train_mae_horizon_mean': np.float64(7.417960157318693),
 'train_mae_horizon_std': np.float64(1.265641796374814

In [11]:
run_id

'770d909ddaf74e91ace9011fd3c09eb9'

In [12]:
preprocessor = trainer.preprocessor
preprocessor.is_fitted

True

## 3. Evaluation

In [13]:
evaluator = ModelEvaluator()

INFO:src.model.evaluator:ModelEvaluator initialized


In [14]:
results = evaluator.evaluate_model(model, X_test, y_test, preprocessor)

INFO:src.model.evaluator:Starting complete model evaluation on 11 samples...
INFO:src.model.evaluator:Making predictions...
INFO:src.model.evaluator:Calculating overall performance metrics...
INFO:src.model.evaluator:Overall metrics calculated: RMSE=175.24, MAE=112.72, R²=0.725
INFO:src.model.evaluator:Calculating per-horizon performance metrics...
INFO:src.model.evaluator:Per-horizon metrics calculated for 24 horizons
INFO:src.model.evaluator:Calculating horizon statistics...
INFO:src.model.evaluator:Horizon statistics calculated: RMSE range [9.08, 344.61]
INFO:src.model.evaluator:Model evaluation completed successfully
INFO:src.model.evaluator:Overall performance: RMSE=175.24, MAE=112.72, R²=0.725
INFO:src.model.evaluator:Best horizon (RMSE): 6 (9.08)
INFO:src.model.evaluator:Worst horizon (RMSE): 22 (344.61)


In [15]:
print("\n" + evaluator.get_evaluation_summary(results))


Model Evaluation Summary
Test Samples: 11
Forecast Horizons: 24
Feature Count: 31

Overall Performance:
- RMSE: 175.24 kW
- MAE:  112.72 kW
- R²:   0.725
- MAPE: 187517511997.0%

Predictions within error bounds:
- ±10%: 8.7%
- ±20%: 14.4%
- ±30%: 22.3%

Horizon Analysis:
- Best performing horizon (RMSE): 6h
- Worst performing horizon (RMSE): 22h
- RMSE range: 9.08 - 344.61 kW
- Mean RMSE: 147.97 ± 93.87 kW


## 4. Registry

In [16]:
registry = ModelRegistry()

INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.model.registry:ModelRegistry initialized with MLflow backend


In [17]:
registry.list_models()

INFO:src.model.registry:Listing all registered models...
INFO:src.model.registry:Found 1 registered models


[{'name': 'solar-forecasting-prod',
  'description': 'Solar forecasting model package created on 2025-07-29 08:40:46',
  'creation_timestamp': 1753778446235,
  'last_updated_timestamp': 1753794943218,
  'total_versions': 7,
  'current_stages': {'Production': '7', 'Archived': '6'}}]

In [18]:
version = registry.register_model_package(
    model, 
    preprocessor, 
    "solar-forecasting-prod",
    description="XGBoost time series model with optimal parameters",
    tags={
        "algorithm": "xgboost",
        "horizon": "24h",
        "features": str(len(preprocessor.get_feature_names())),
        "test_rmse": str(round(results['overall']['rmse'], 2))
    },
    run_id=run_id,  
)

INFO:src.model.registry:Registering model package: solar-forecasting-prod
INFO:src.model.registry:Using existing registered model: solar-forecasting-prod
INFO:src.data.preprocessor:Preprocessor saved to: /tmp/tmpvpswi60o.pkl
2025/07/29 13:18:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/29 13:18:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'solar-forecasting-prod' already exists. Creating a new version of this model...
2025/07/29 13:18:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: solar-forecasting-prod, version 8
Created version '8' of model 'solar-forecasting-prod'.
INFO:src.model.registry:Successfully registered model package solar-forecasting-prod version 8
INFO:src.model.registry:Model URI: models:/so

🏃 View run xgboost_training_20250729_131844 at: http://localhost:5000/#/experiments/3/runs/770d909ddaf74e91ace9011fd3c09eb9
🧪 View experiment at: http://localhost:5000/#/experiments/3


In [19]:
registry.transition_to_production("solar-forecasting-prod", version)

INFO:src.model.registry:Transitioning model solar-forecasting-prod v8 to Production
INFO:src.model.registry:Archiving existing version 7 from Production
INFO:src.model.registry:Successfully transitioned model to Production


## 5. Production Loading

In [20]:
prod_model, prod_preprocessor = registry.load_production_model("solar-forecasting-prod")

INFO:src.model.registry:Loading model package: solar-forecasting-prod
INFO:src.model.registry:Loading from stage: Production
INFO:src.model.registry:Loading model from URI: models:/solar-forecasting-prod/Production


INFO:src.model.registry:Looking for preprocessor at: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Found preprocessor file: preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Loading preprocessor from: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.data.preprocessor:Preprocessor loaded from: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Successfully loaded model package solar-forecasting-prod
INFO:src.model.registry:Model type: MultiOutputRegressor
INFO:src.model.registry:Preprocessor features: 31


In [21]:
print("Production model loaded successfully!")
print(f"   Model type: {type(prod_model).__name__}")
print(f"   Preprocessor features: {len(prod_preprocessor.get_feature_names())}")
print(f"   Forecast horizon: {prod_preprocessor.forecast_horizon}h")
test_features = X_test.drop('DATE_TIME', axis=1).iloc[:1]
original_pred = model.predict(test_features)
production_pred = prod_model.predict(test_features)
print(f"   Prediction consistency check: {np.allclose(original_pred, production_pred)}")

Production model loaded successfully!
   Model type: MultiOutputRegressor
   Preprocessor features: 31
   Forecast horizon: 24h
   Prediction consistency check: True


## 6. Test Batch

In [22]:
# Initialize registry for batch service
registry = ModelRegistry()

# Load production model (simulating batch service)
model, preprocessor = registry.load_production_model("solar-forecasting-prod")

# Test midnight prediction for a specific date
test_date = "2020-06-15"
print(f"   Predicting for date: {test_date}")
print(f"   Scenario: Predict {test_date} 01:00-24:00 at {test_date} 00:00")

try:
    # Operational midnight prediction
    features = preprocessor.prepare_midnight_prediction(data, test_date)
    forecast_24h = model.predict(features.drop('DATE_TIME', axis=1))
    
    print("Midnight prediction successful!")
    print(f"   Features shape: {features.shape}")
    print(f"   Forecast shape: {forecast_24h.shape}")
    print(f"   Peak predicted power: {forecast_24h[0].max():.1f} kW at hour {np.argmax(forecast_24h[0])+1}")
    print(f"   Total predicted energy: {forecast_24h[0].sum():.1f} kWh")
    print(f"   Daylight hours energy (6-18): {forecast_24h[0][6:19].sum():.1f} kWh")
    
    # Show first few hours of forecast
    print(f"\n   First 6 hours forecast:")
    for i in range(6):
        print(f"     Hour {i+1:2d}: {forecast_24h[0][i]:6.1f} kW")
        
    print(f"\nOperational batch prediction test: PASSED")
    
except Exception as e:
    print(f"Operational batch prediction test: FAILED")
    print(f"   Error: {str(e)}")

INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.utils.mlflow_utils:MLflow tracking URI set to: http://localhost:5000
INFO:src.utils.mlflow_utils:MLflow server will handle S3 artifacts automatically
INFO:src.model.registry:ModelRegistry initialized with MLflow backend
INFO:src.model.registry:Loading model package: solar-forecasting-prod
INFO:src.model.registry:Loading from stage: Production
INFO:src.model.registry:Loading model from URI: models:/solar-forecasting-prod/Production


INFO:src.model.registry:Looking for preprocessor at: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Found preprocessor file: preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Loading preprocessor from: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.data.preprocessor:Preprocessor loaded from: /workspaces/solar-forecasting-mlops/artifacts/preprocessor_770d909ddaf74e91ace9011fd3c09eb9.pkl
INFO:src.model.registry:Successfully loaded model package solar-forecasting-prod
INFO:src.model.registry:Model type: MultiOutputRegressor
INFO:src.model.registry:Preprocessor features: 31
INFO:src.data.preprocessor:Preparing midnight prediction for 2020-06-15
INFO:src.data.preprocessor:Creating historical lag features...
INFO:src.data.preprocessor:Created 5 lag features and 2 rolling features
INFO:src.data.preprocessor:Creating temporal features for fu

   Predicting for date: 2020-06-15
   Scenario: Predict 2020-06-15 01:00-24:00 at 2020-06-15 00:00
Midnight prediction successful!
   Features shape: (2, 32)
   Forecast shape: (2, 24)
   Peak predicted power: 601.8 kW at hour 9
   Total predicted energy: 9322.8 kWh
   Daylight hours energy (6-18): 5297.0 kWh

   First 6 hours forecast:
     Hour  1:  431.1 kW
     Hour  2:  487.5 kW
     Hour  3:  189.2 kW
     Hour  4:  179.0 kW
     Hour  5:  405.3 kW
     Hour  6:  547.0 kW

Operational batch prediction test: PASSED
